# Artic Skyze

In [2]:
from datetime import datetime
import time
import ystockquote

from arctic import Arctic
import collections
import pandas
import pandas as pd
import pprint


from trading_ig import IGService
#from trading_ig.config import config

# Connect to Local MONGODB

arctic = Arctic('localhost')

In [3]:
print(type(arctic))
print(arctic)

<class 'arctic.arctic.Arctic'>
<Arctic at 0x1033f9f98, connected to MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, maxpoolsize=4, sockettimeoutms=600000, connecttimeoutms=2000, serverselectiontimeoutms=30000)>


In [4]:
arctic.list_libraries()

['SKYZE', 'NASDAQ']

In [5]:
# Create a VersionStore library
arctic.delete_library('jblackburn.stocks')
arctic.delete_library('NASDAQ')
arctic.initialize_library('SKYZE')

Library created, but couldn't enable sharding: no such command: 'enablesharding', bad cmd: '{ enablesharding: "arctic" }'. This is OK if you're not 'admin'


In [6]:
arctic.list_libraries()

['SKYZE']

# get stock history YAHOO

In [7]:
def get_stock_history(ticker, start_date, end_date):
    data = ystockquote.get_historical_prices(ticker, start_date, end_date)
    df = pandas.DataFrame(collections.OrderedDict(sorted(data.items()))).T
    df = df.convert_objects(convert_numeric=True)
    return df

In [8]:
stocks = arctic['SKYZE']
print(type(stocks))
print(stocks)

<class 'arctic.store.version_store.VersionStore'>
    <ArcticLibrary at 0x1033f9c18, arctic.SKYZE>
        <Arctic at 0x1033f9f98, connected to MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, maxpoolsize=4, sockettimeoutms=600000, connecttimeoutms=2000, serverselectiontimeoutms=30000)>


In [ ]:
aapl = get_stock_history('aapl', '2015-01-01', '2015-02-01')

# Get IG Markets

In [14]:
import os
#print (os.environ)
acc_number = os.environ['IG_SERVICE_ACC_NUMBER']
password = os.environ['IG_SERVICE_PASSWORD']
api_key = "38b13896c1dbbc27919c2e9198ac2bdb3cde2887" #os.environ['IG_SERVICE_API_KEY']
username = os.environ['IG_SERVICE_USERNAME']
acc_type = "LIVE" #os.environ['IG_SERVICE_ACC_TYPE']
print(username); print(password); print(api_key); print(acc_type)

# Crete the session
ig_service = IGService(username, password, api_key, acc_type)
print(ig_service)
ig_service.create_session()


synchronicity
Eldership88*
38b13896c1dbbc27919c2e9198ac2bdb3cde2887
LIVE


{'accountInfo': {'available': 132.0,
  'balance': 132.0,
  'deposit': 0.0,
  'profitLoss': 0.0},
 'accountType': 'CFD',
 'accounts': [{'accountId': 'NZ3B3',
   'accountName': 'CFD',
   'accountType': 'CFD',
   'preferred': True}],
 'clientId': '102144929',
 'currencyIsoCode': 'AUD',
 'currencySymbol': 'A$',
 'currentAccountId': 'NZ3B3',
 'dealingEnabled': True,
 'hasActiveDemoAccounts': True,
 'hasActiveLiveAccounts': True,
 'lightstreamerEndpoint': 'https://apd.marketdatasystems.com',
 'reroutingEnvironment': None,
 'timezoneOffset': 10,
 'trailingStopsEnabled': False}

In [16]:
account_info = ig_service.switch_account(acc_number, False) # not necessary
print(account_info)


AttributeError: 'IGService' object has no attribute '_set_headers'

In [17]:
open_positions = ig_service.fetch_open_positions()
print("open_positions:\n%s" % open_positions)

open_positions:
Empty DataFrame
Columns: [lotSize, epic, controlledRisk, percentageChange, netChange, expiry, marketStatus, dealSize, openLevel, contractSize, currency, updateTime, direction, scalingFactor, delayTime, streamingPricesAvailable, offer, instrumentName, high, trailingStopDistance, limitLevel, low, createdDate, dealId, stopLevel, trailingStep, instrumentType, bid]
Index: []

[0 rows x 28 columns]


In [19]:
epic = 'CS.D.EURUSD.MINI.IP'
resolution = 'D'
num_points = 10
response = ig_service.fetch_historical_prices_by_epic_and_num_points(epic, resolution, num_points)
df_ask = response['prices']['ask']
print("ask prices:\n%s" % df_ask)
      

ask prices:
                        Open     High      Low    Close
DateTime                                               
2017:09:19-00:00:00  1.19603  1.20073  1.19226  1.19858
2017:09:20-00:00:00  1.19859  1.20228  1.19607  1.19971
2017:09:21-00:00:00  1.19972  1.20356  1.18621  1.19254
2017:09:22-00:00:00  1.19255  1.20053  1.19104  1.19687
2017:09:23-00:00:00  1.19684  1.19845  1.19372  1.19499
2017:09:25-00:00:00  1.19061  1.19374  1.18624  1.18795
2017:09:26-00:00:00  1.18796  1.18892  1.17807  1.17816
2017:09:27-00:00:00  1.17822  1.18114  1.17173  1.17263
2017:09:28-00:00:00  1.17264  1.17962  1.17188  1.17953
2017:09:29-00:00:00  1.17955  1.18050  1.17729  1.17959
